In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import numpy as np
import collections as col
import scipy.sparse as sp
from scipy.sparse import lil_matrix


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lvallet/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [78]:
data_test = pd.read_csv("./arxiv_articles.csv", sep="|",usecols=[1,3,4])
#nrow permet de selectionner un certain nb de docs

In [79]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
#fonctions appliquées sur chaque document pour nettoyage
b = lambda new : " ".join([ps.stem(i) for i in re.sub("[^a-zA-Z]", " " ,new.lower()).split() if i not in stop_words]).split()
#a = lambda x : col.Counter(re.findall('\w+',x))

In [80]:
data_test['cleaned']=data_test['summary'].apply(b)
#data_test['dict']=data_test['cleaned'].apply(a)

In [81]:
def calcul_tf(d):
    term_counts = pd.Series(d).value_counts()
    max_mot = np.max(term_counts)
    return 0.5 + 0.5 * (term_counts / max_mot)
def compute_idf(D):
    N = len(D)
    all_terms = pd.concat(D)
    nt = all_terms.index.value_counts() # The number of documents containing the term "t"
    return np.log(N / nt)
def compute_tf_idf_document(tf_document, idf):
    return tf_document * np.array([idf[i] for i in tf_document.index])
#calcul le tf idf pour 1 document
def compute_tf_idf_corpus(D):
    term_freq = [calcul_tf(d) for d in D]
    idf = compute_idf(term_freq)
    return [compute_tf_idf_document(d, idf) for d in term_freq]
#prend en entree une colonne panda series et return un 

In [82]:
tf_idf = compute_tf_idf_corpus(data_test.loc[:10000,"cleaned"])
#liste avec comme indice le numéro du document et pour chaque document un panda series 
#avec mot comme index et tf_idf pour le mot comme values
#array de nos mots
all_mot = pd.concat(tf_idf).index.drop_duplicates() #attention il n'additionne pas si deux fois le même mot
#print((tf_idf.append(self=tf_idf,))

In [113]:
data=pd.DataFrame(tf_idf).fillna(0)
dense_matrix = np.array(data.as_matrix())
sparse_matrix = sp.csr_matrix(dense_matrix)
print(np.shape(dense_matrix))

#c = lambda x : x if x not 'NaN'

KeyboardInterrupt: 

In [120]:
#affiche le tf_idf d'un mot : group dans doc 1
print(sparse_matrix[:,all_mot.get_loc('cv')])
print(all_mot.get_loc('group'))
print(dense_matrix[0,0])
print(all_mot.value_counts())

  (167, 0)	4.7318997123465705
  (475, 0)	7.097849568519855
  (624, 0)	5.407885385538937
8
0.0
cv        1
esorex    1
wife      1
masato    1
raugel    1
         ..
lisa      1
zg        1
faulk     1
rao       1
itak      1
Length: 19809, dtype: int64


In [106]:
#print(sparse_matrix)
print(data_test.loc[1,'cleaned'])
print(sparse_matrix[1,:])

['han', 'grauert', 'die', 'septemb', 'articl', 'review', 'life', 'mathemat', 'recal', 'detail', 'major', 'accomplish']
  (0, 97)	5.74470446417679
  (0, 882)	3.4610473810682625
  (0, 4243)	2.9978342712249972
  (0, 4339)	6.571383037361257
  (0, 6992)	9.210440366976517
  (0, 7231)	7.418680897748461
  (0, 9644)	4.22683374526818
  (0, 10122)	3.236630755107255
  (0, 10298)	4.01194333571069
  (0, 14315)	6.214708093422525
  (0, 14704)	3.184574393151202
  (0, 15536)	5.809242985314361


In [115]:
tf_idf[0].loc['polish']

7.418680897748461